In [58]:
import requests
from bs4 import BeautifulSoup
import re
from pprint import pprint
from sqlalchemy import create_engine
from pprint import pprint
import pandas as pd
import csv
from sqlalchemy import create_engine
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
import torch
import torch.nn.functional as F

In [11]:
print(df.info())
print(row1)
df_copy = df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12580 entries, 0 to 12579
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   GeoLocationRPT   12580 non-null  object
 1   broj_soba_s      12580 non-null  object
 2   grejanje_s       11943 non-null  object
 3   grad_s           12580 non-null  object
 4   lokacija_s       12580 non-null  object
 5   mikrolokacija_s  12580 non-null  object
 6   kvadratura_d     12580 non-null  object
 7   sprat_od_s       11883 non-null  object
 8   ulica_t          11629 non-null  object
 9   cena_d           12580 non-null  object
 10  Uknjizen         12580 non-null  int64 
 11  Klima            12580 non-null  int64 
 12  Lift             12580 non-null  int64 
 13  Garaza           12580 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 1.3+ MB
None
('44.738500,20.489078', '1.0', 'EG', 'Beograd', 'Opština Voždovac', 'Jajinci', '27.53', '2', 'Umetnička 1. prilaz',

In [56]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('db_prodaja_stanova_Beograd.db')

# Create a cursor object
cursor = conn.cursor()
df = pd.read_sql_query("SELECT * FROM mytable", conn)

# Execute a SELECT query
cursor.execute("SELECT * FROM mytable")

X_data = list()
Y_data = list()

# Fetch and print rows one by one
row1 = cursor.fetchone()
row = cursor.fetchone()
while row is not None:
    # print(row)
    latitude, longitude = coordinates.split(',')
    latitude = float(latitude)
    longitude = float(longitude)

    number_rooms = row[1]
    number_rooms = float(number_rooms) if number_rooms != '5+' else 5.0

    kvadratura = float(row[6])

    floor = row[7]
    if floor == True:
        floor = float(floor)
    # else:
    #     break
    legal = float(row[10])
    air_c = float(row[11])
    elevator = float(row[12])
    garage = float(row[13])
    # , legal, air_c, elevator, garage

    row_x_data = torch.Tensor([latitude,longitude, number_rooms, kvadratura, legal, elevator, garage])
    price = float(row[9])
    X_data.append(row_x_data)
    Y_data.append(price)

    row = cursor.fetchone()

# Close the connection
conn.close()

In [38]:

df.dropna(subset=["sprat_od_s"], inplace=True)
print(X_data)

tensor([[44.7385, 20.4891,  2.0000,  ...,  1.0000,  1.0000,  1.0000],
        [44.7385, 20.4891,  0.5000,  ...,  0.0000,  1.0000,  0.0000],
        [44.7385, 20.4891,  4.0000,  ...,  1.0000,  1.0000,  0.0000],
        ...,
        [44.7385, 20.4891,  3.5000,  ...,  1.0000,  1.0000,  0.0000],
        [44.7385, 20.4891,  3.0000,  ...,  1.0000,  1.0000,  0.0000],
        [44.7385, 20.4891,  2.0000,  ...,  1.0000,  1.0000,  1.0000]])


In [52]:

print(row1)
coordinates = row1[0]

latitude, longitude = coordinates.split(',')
latitude = float(latitude)
longitude = float(longitude)

number_rooms = row1[1]
number_rooms = float(number_rooms) if number_rooms != '5+' else 5.0

kvadratura = float(row1[6])

floor = row1[7]
if floor == True:
    floor = float(floor)
# else:
#     break
legal = float(row1[10])
air_c = float(row1[11])
elevator = float(row1[12])
garage = float(row1[13])

# , legal, air_c, elevator, garage
row_end_proces = [latitude,longitude, number_rooms, kvadratura]
cena = float(row1[9])

print(row_end_proces)

('44.738500,20.489078', '1.0', 'EG', 'Beograd', 'Opština Voždovac', 'Jajinci', '27.53', '2', 'Umetnička 1. prilaz', '49280.0', 0, 0, 1, 0)
[44.7385, 20.489078, 1.0, 27.53]


In [61]:
X_data = F.normalize(X_data, dim=0)
print(X_data)

tensor([[0.0089, 0.0089, 0.0061, 0.0057],
        [0.0089, 0.0089, 0.0015, 0.0020],
        [0.0089, 0.0089, 0.0122, 0.0092],
        ...,
        [0.0089, 0.0089, 0.0107, 0.0060],
        [0.0089, 0.0089, 0.0092, 0.0111],
        [0.0089, 0.0089, 0.0061, 0.0054]])


In [63]:
class LinearRegression(torch.nn.Module):

    def __init__(self, inputS, outputS): 
        super(LinearRegression, self).__init__() 
        self.linear = torch.nn.Linear(inputS, outputS)
    def forward(self, x): 

        predict_y = self.linear(x) 
        return predict_y 

In [62]:
linear_model = LinearRegression(4,1)
define_criterion = torch.nn.MSELoss(size_average=False)
SGD_optimizer = torch.optim.SGD(linear_model.parameters(), lr=0.00001)

C:\Users\busketi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [64]:
for epoch in range(500): 


    predict_y = linear_model(X_data) 
    loss = define_criterion(predict_y, Y_data) 


    SGD_optimizer.zero_grad() 
    loss.backward() 

    SGD_optimizer.step() 
    print('epoch {}, loss function {}'.format(epoch, loss.item()))

epoch 0, loss function 969716709457920.0
epoch 1, loss function 718102425763840.0
epoch 2, loss function 577190857015296.0
epoch 3, loss function 498275664789504.0
epoch 4, loss function 454079947997184.0
epoch 5, loss function 429328017915904.0
epoch 6, loss function 415465071443968.0
epoch 7, loss function 407700273889280.0
epoch 8, loss function 403350579314688.0
epoch 9, loss function 400913386700800.0
epoch 10, loss function 399547285110784.0
epoch 11, loss function 398781002547200.0
epoch 12, loss function 398350734065664.0
epoch 13, loss function 398108437512192.0
epoch 14, loss function 397971568984064.0
epoch 15, loss function 397893722701824.0
epoch 16, loss function 397848893980672.0
epoch 17, loss function 397822553751552.0
epoch 18, loss function 397806581841920.0
epoch 19, loss function 397796448403456.0
epoch 20, loss function 397789536190464.0
epoch 21, loss function 397784435916800.0
epoch 22, loss function 397780375830528.0
epoch 23, loss function 397776852615168.0
ep

In [ ]:
df.dropna(subset=["sprat_od"])

In [8]:
coordinates = 

# Split the string into two parts
parts = coordinates.split(',')

# Convert the parts into real numbers
latitude = float(parts[0])
longitude = float(parts[1])

# Print the converted values
print("Latitude:", latitude)
print("Longitude:", longitude)

(12580, 14)

In [33]:
df["cena_d"].isnull().any()

False